In [1]:
import xgboost as xgb
import numpy as np
import pandas as pd# Used for Regression Modelling
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle

In [21]:
data = pd.read_csv('./data/merged_mldata2020v2.csv',delimiter = ',')
with open(f'score_predictor.pkl', 'rb') as f:
    model = pickle.load(f)

outputs = []

games = data.drop(columns =['vscore', 'hscore','winner','ou',"sWinner","fspread","fscore","vospread","vcspread"])
games = pd.DataFrame(games)
weekfilter = games['week'] == 1

games = games[weekfilter]
print(games)
columnheads = ['temp','wind_mph','vdflg','hdflg','divgame','nsite','hospread','ouopen','hcspread','ouclose','vTOTAL.DVOA','vTOTAL.RNK','vOFF.RNK','vOFF.DVOA','vDEF.RNK','vDEF.DVOA','vST.RNK','vST.DVOA','hTOTAL.DVOA','hTOTAL.RNK','hOFF.RNK','hOFF.DVOA','hDEF.RNK','hDEF.DVOA','hST.RNK','hST.DVOA','vtsw','vtsl','vtst','htsw','htsl','htst','vtw','vtl','vtt','vts','htw','htl','htt','hts']

games = np.hsplit(games,[3])

tw = games[0]
pdata = games[1]



input_variables = pd.DataFrame(pdata, columns = columnheads,dtype=float)

#prediction = model.predict(input_variables)

input_variables["prediction"] = model.predict(input_variables)

#print(prediction)
#print(input_variables)

output_info = input_variables[["hcspread","ouclose","prediction"]]
output_info = tw.join(output_info)

#output_info["pteam"] = output_info["vteam"] if output_info["vteam"] =="-1" else output_info["hteam"]
output_info["prediction"] = np.where(output_info["prediction"] < 0, output_info["vteam"], output_info["hteam"])
print(output_info)
#for i in games.iterrows():
    #prediction = {'vteam':i['vteam'],'hteam':i['hteam']}
    #input_variables = x.drop(columns=['week','vteam','hteam'])
    #prediction['outcome'] = model.predict(input_variables)[0]
    #print(i['vteam'])


#print(x)
#print(teams)

    week vteam hteam  temp  wind_mph  vdflg  hdflg  divgame  nsite  hospread  \
0      1   HOU    KC    58         6     -1     -1        0      0      -9.5   
1      1   MIA    NE    73         6     -1     -1        1      0      -7.0   
2      1   CLE   BAL    75         5     -1     -1        1      0      -8.5   
3      1   NYJ   BUF    65        12     -1     -1        1      0      -6.0   
4      1    LV   CAR    79         3     -1     -1        0      0       1.5   
5      1   SEA   ATL    72         0     -1     -1        0      0      -1.0   
6      1   PHI   WAS    76         4     -1     -1        1      0       6.5   
7      1   CHI   DET    72         0     -1     -1        1      0      -1.0   
8      1   IND   JAX    86         8     -1     -1        1      0       6.5   
9      1    GB   MIN    72         0     -1     -1        1      0      -3.0   
10     1   LAC   CIN    83         4     -1     -1        0      0       3.0   
11     1   ARI    SF    71         6    